In [1]:
import random
import numpy as np
import tensorflow as tf

In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8')

In [3]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()
text = text[300000:800000]

In [4]:
characters = sorted(set(text))
# print(characters)


char_to_index = dict((c, i) for i, c in enumerate(characters))
# print(char_to_index)


index_to_char = dict((i, c) for i, c in enumerate(characters))
# print(index_to_char)


In [5]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_char = []

# sentences will have sentences and
# next_char will try to predict next word in sentences 

In [6]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])
    
    # print(next_char)
    
    #simply this loop is breaking all the words in chars
    #Eg :=.   
    #    next_char = ['Hello', 'ello,', 'llo, ', 'lo, h', 'o,
    #     ho', ', how', ' how ', 'how a', 'ow ar', '
    #     w are', ' are ', 'are y', 're yo', 'e you', ' you', 'you ', 
    #    'ou to', 'u tod', ' toda', 'today']



In [7]:
#  code is creating a data representation for 
# training a neural network. It encodes sequences of 
# characters (x) and the next character after each sequence (y) 
# in a binary format, where 1 indicates the presence of a 
# character at a specific position.

x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=bool)
y = np.zeros((len(sentences),
              len(characters)), dtype=bool)


for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1
    

    


# making neural network

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

In [9]:
model = Sequential()
#sequential?? a simple stack layer where 
# each layer has one input tensor and one output tensor



model.add(LSTM(128,input_shape=(SEQ_LENGTH,len(characters))))
#adding a lstm layer with 128neurons to model

model.add(Dense(len(characters)))
# This specifies a dense layer with a number of units
# equal to the length of the characters
# character=['\n', ' ', '!', '$', '&', "'", ',', '-', '.',
#            '3', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 
#            'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
# The number of units in the dense layer often 
# corresponds to the number of classes or unique
# elements in the output.


model.add(Activation('softmax'))
# it takes a set of numbers and converts 
# them into a probability distribution,
# meaning it makes them look like probabilities.

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.01))

model.fit(x, y, batch_size=256, epochs=4)

Epoch 1/4
651/651 [==============================] - 31s 48ms/step - loss: 2.6889
Epoch 2/4
651/651 [==============================] - 32s 49ms/step - loss: 2.2997
Epoch 3/4
651/651 [==============================] - 31s 48ms/step - loss: 2.1830
Epoch 4/4
651/651 [==============================] - 31s 48ms/step - loss: 2.0928


In [11]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# generating

In [12]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [13]:
print(generate_text(300, 0.2))
print(generate_text(300, 0.4))
print(generate_text(300, 0.5))
print(generate_text(300, 0.6))
print(generate_text(300, 0.7))
print(generate_text(300, 0.8))

ilt be satisfied.

juliet:
indeed, i neve me the south the beather will his the sored the forther and the fored he meatere the heard,
and sould the fore ou the king of the sone.

rome if or is mare the will so for the heard the shere,
the south the sone the hear of the court.

hereri:
and i arderde:
and so my more to me thou have the grou
s thou rescue him from foul despair?

bour and is come:
of there sould so ford if the bist the singed is thou see the sane.

areer:
eristle:
hour the love, thou hear s, but the soust the king.

lereice:
here mare of tore ard in the seant the ingering ingerore.

hareave:
i wart of all of uree the prould the singer in the come of in;
and i 
 out
and makes himself an artificial nigh, of his dead, i seans
enot hou the gover hath all then meard in wore the gee to gake of theres,
i will to fored fare ous nost, and the med to here,
in ming his with the ore i bered that hish ant sour has hald cone.

burien:
thou hime the bence that the sien thou dave,
and wit